In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from simpletransformers.classification import MultiLabelClassificationModel

In [2]:
train = pd.read_csv('raw_data/train.csv')
test = pd.read_csv('raw_data/nlp_test.csv')
sub = pd.read_csv('raw_data/sample_submission.csv')

In [3]:
train.head()

,ID,category_A,category_B,category_C,category_D,category_E,category_F,Question Sentence
0,QC0000000001,0,1,0,0,0,0,病情描述：病人是典型的“三高”，想吃拜阿司匹林做为预防用药，但是出现过敏症状。曾经治疗情况和...
1,QC0000000002,0,1,0,0,0,0,"病情描述：我父亲78岁,小脑梗塞，表现左眼双视，经住院输液治疗现在恢复一月余，现在看人不重影..."
2,QC0000000003,0,1,0,0,0,0,病情描述：医生您好！我想问一下，我妈脑梗塞今天住院十一天了，情况好转，可以下床活动和清楚表达...
3,QC0000000004,1,0,0,0,0,0,停药反跳的血压是身体真实状况吗?
4,QC0000000005,0,1,0,0,0,0,病情描述：医生：银杏叶片生产厂很多，价格差别很大，应怎样选择？曾经治疗情况和效果：头部紧束感...


In [4]:
test.head()

,ID,Question Sentence
0,QCT1000000001,乙肝抗体多久打一次乙肝抗体多久打一次，戊型肝炎的传播途径是怎样的，伴侣中有一个是乙肝携带，不...
1,QCT1000000002,十日观察法我没被狗咬伤，我想问问，狂犬病不是有十日观察法吗，那如果说狗咬人时狗得了狂犬病，可...
2,QCT1000000003,我的乙肝大三阳十年了，现在能治的好了吗沈阳肝病携带者能治好，我的乙肝大三阳十年了，现在能治的...
3,QCT1000000004,治疗丙肝哪里治疗最好丙肝的预防与治疗，治疗丙肝哪里治疗最好，丙肝表现症状是什么呢？丙肝有哪些...
4,QCT1000000005,婚检乙肝上千倍婚检乙肝上千倍，我父亲肝硬化失代偿期，有单个结节癌变。有乙肝病史已经射频手术，...


In [5]:
sub.head()

,ID,category_A,category_B,category_C,category_D,category_E,category_F
0,QCT1000000001,1,1,0,0,1,0
1,QCT1000000002,0,1,0,1,1,1
2,QCT1000000003,0,1,1,0,1,0
3,QCT1000000004,1,1,0,1,0,0
4,QCT1000000005,0,1,0,0,1,1


In [6]:
train.shape, test.shape

((5000, 8), (3000, 2))

In [7]:
train['Question Sentence'].apply(lambda x: len(x)).describe()

count    5000.000000
mean      110.758800
std       127.906681
min         4.000000
25%        42.000000
50%        77.000000
75%       132.000000
max      2093.000000
Name: Question Sentence, dtype: float64

In [8]:
test['Question Sentence'].apply(lambda x: len(x)).describe()

count    3000.000000
mean      144.382667
std       224.694973
min        19.000000
25%        54.000000
50%        82.000000
75%       134.000000
max      3095.000000
Name: Question Sentence, dtype: float64

In [9]:
train_df = train[['Question Sentence']].copy()
train_df['labels'] = train[['category_A','category_B','category_C','category_D','category_E','category_F']].values.tolist()
train_df.columns = ['text', 'labels']

train_df.head()

,text,labels
0,病情描述：病人是典型的“三高”，想吃拜阿司匹林做为预防用药，但是出现过敏症状。曾经治疗情况和...,"[0, 1, 0, 0, 0, 0]"
1,"病情描述：我父亲78岁,小脑梗塞，表现左眼双视，经住院输液治疗现在恢复一月余，现在看人不重影...","[0, 1, 0, 0, 0, 0]"
2,病情描述：医生您好！我想问一下，我妈脑梗塞今天住院十一天了，情况好转，可以下床活动和清楚表达...,"[0, 1, 0, 0, 0, 0]"
3,停药反跳的血压是身体真实状况吗?,"[1, 0, 0, 0, 0, 0]"
4,病情描述：医生：银杏叶片生产厂很多，价格差别很大，应怎样选择？曾经治疗情况和效果：头部紧束感...,"[0, 1, 0, 0, 0, 0]"


In [10]:
eval_df = train_df[4000:]
train_df = train_df[:4000]

train_df.shape, eval_df.shape

((4000, 2), (1000, 2))

In [11]:
args = {
    'num_train_epochs': 5,
    'fp16': False,
    'no_save': True,
    'overwrite_output_dir': True,
    'max_seq_length': 128,
    'evaluate_during_training': True,
    'evaluate_during_training_verbose': True
}

In [12]:
model = MultiLabelClassificationModel('bert', 
                                      'hfl/chinese-roberta-wwm-ext', 
                                      num_labels=6, 
                                      args=args)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification

In [13]:
model.train_model(train_df, eval_df=eval_df)

In [14]:
results = model.predict([text for text in test['Question Sentence']])

In [15]:
sub['category_A'] = np.array(results[0])[:, 0]
sub['category_B'] = np.array(results[0])[:, 1]
sub['category_C'] = np.array(results[0])[:, 2]
sub['category_D'] = np.array(results[0])[:, 3]
sub['category_E'] = np.array(results[0])[:, 4]
sub['category_F'] = np.array(results[0])[:, 5]

sub.head()

,ID,category_A,category_B,category_C,category_D,category_E,category_F
0,QCT1000000001,1,0,0,1,0,0
1,QCT1000000002,0,1,0,0,0,0
2,QCT1000000003,0,1,0,1,0,0
3,QCT1000000004,1,1,0,0,0,0
4,QCT1000000005,0,1,0,0,0,0


In [16]:
sub.to_csv('roberta_wwm_ext_baseline.csv', index=False)